# Connecting to Database Using SQLAlchemy

### Connection through a public user with restricted access
### Allows user to search for a book and get results showing how many books are available and in which store

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text
from urllib.parse import unquote

# using pandas for displaying result
import pandas as pd
from IPython.display import display

# set up connection string variables using the user Public Store Access
# this user is set up to have minimal access to the database and is intended to be used in external applications such as this
# in this case it can only access executing the stored procedure "SearchBook" in the laboration database and nothing else
driver = "ODBC Driver 17 for SQL Server"
server_name   = "localhost"
database_name = "laboration"
username = "Public Store Access"
password = "123"

connection_string = f"Driver={driver};Server={server_name};Database={database_name};UID={username};PWD={password};"
url_string = URL.create("mssql+pyodbc", query = dict(odbc_connect = connection_string))

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try: # attempt to connect using the url string
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name} database!') # print a message if successful
except Exception as err: # print an error message if unsuccessful
    print('Error while connecting to database:\n')
    print(err)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=Driver=ODBC+Driver+17+for+SQL+Server;Server=localhost;Database=laboration;UID=Public+Store+Access;PWD=123; 

Successfully connected to laboration database!


In [2]:
def execute_sp(title, connection):
    """function for executing the SearchBook SP in laboration database, returns books matching title parameter and how many there are in each store"""
    query = text("""EXECUTE SearchBook :title""") # injection safe way of handling variable value, query takes a :title parameter
    results = connection.execute(query, {'title': f'%{title}%'}).fetchall() # and here the :title parameter is given the value of title
    return results

def search_book():
    # let user input a book title
    title = input("Search for a book: ").strip()

    with engine.connect() as connection:
        results = execute_sp(title, connection) # book search results fetched
        df = pd.DataFrame(results, columns = ["Store", "Title", "Format", "Amount"]) # results stored as dataframe
        print(f"Search for \"{title}\" gave {len(df)} results.")
        if len(df) > 0:
            display(df) # dataframe displayed in a neat formatted manner

In [3]:
search_book()

Search for "Embassytown" gave 6 results.


,Store,Title,Format,Amount
0,Bookstore Gothenburg Central,Embassytown,Hardcover,12
1,Bookstore Gothenburg Central,Embassytown,Paperback,1
2,Bookstore Gothenburg North,Embassytown,Paperback,1
3,Bookstore Gothenburg North,Embassytown,Hardcover,0
4,Bookstore Lerum,Embassytown,Paperback,3
5,Bookstore Lerum,Embassytown,Hardcover,2


In [4]:
search_book() # can also search for parts of words, and not case-sensitive

Search for "BaSsY" gave 6 results.


,Store,Title,Format,Amount
0,Bookstore Gothenburg Central,Embassytown,Hardcover,12
1,Bookstore Gothenburg Central,Embassytown,Paperback,1
2,Bookstore Gothenburg North,Embassytown,Paperback,1
3,Bookstore Gothenburg North,Embassytown,Hardcover,0
4,Bookstore Lerum,Embassytown,Paperback,3
5,Bookstore Lerum,Embassytown,Hardcover,2


In [5]:
search_book()

Search for "45oeghoegh" gave 0 results.


In [6]:
search_book() # cannot use sql injection (and even if it was possible to work around, user has no access to information)

Search for ""; SELECT * FROM company.SaleDetails;" gave 0 results.
